In [1]:
import os
import numpy as np
import pandas as pd
import random
import tensorflow as tf

from datetime import timedelta
from time import time

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, BatchNormalization
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical

In [2]:
# Data Parameters
data_dir = "./data/"
seed = int(time())
random.seed(seed)

num_fan_level = 4
raw = [None] * num_fan_level

train_test = 0.7

# LSTM Parameters
maxlen = 512
batch_size = 16

In [3]:
# Utility
def split_continuous(data, tolerance=0):
    data["time"] = pd.to_datetime(data["time"])
    res = []
    tmp = []
    prev = data.iloc[0][0]
    cnt = 0
    for i in range(1,data.shape[0]):
        if (prev+timedelta(seconds=1)) == data.iloc[i][0]:
            tmp.append(np.array(data.iloc[i][1:3])) # PM2.5, PM10, FAN
        elif cnt < tolerance:
            tmp.append(np.array(data.iloc[i][1:3])) # PM2.5, PM10, FAN
            cnt += 1
        else:
            if tmp != []:
                res.append(tmp)
            tmp = []
            cnt = 0
        prev = data.iloc[i][0]
    return res

In [4]:
# Load Data
raw = [None] * num_fan_level
for file in os.listdir(data_dir):
    file = os.path.join(data_dir,file)
    data = pd.read_csv(file)
    print("Loading {}...  ".format(file),end='')
    for index in range(0,4):
        df = data[data["fan"]==index]
        if df.shape[0] > 0:
            res = split_continuous(df)
            if raw[index] == None:
                raw[index] = res
            else:
                raw[index].extend(res)
        index += 1
    print("Done.")

del df
del file
del data
del index

Loading ./data/2021-01-19oldFilter.csv...  

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Done.
Loading ./data/2021-01-20-newFilter.csv...  Done.
Loading ./data/2021-01-21noFan.csv...  Done.
Loading ./data/2021-01-22noFan.csv...  Done.
Loading ./data/2021-01-23Fan3-night.csv...  Done.
Loading ./data/2021-01-23Fan3.csv...  Done.
Loading ./data/2021-01-24_18.csv...  Done.


In [5]:
print(len(raw[0]))
print(len(raw[1]))
print(len(raw[2]))
print(len(raw[3]))

45
18
7
39


In [6]:
# Train Test Split
def train_test_split(raw, train_test=0.7):
    trainX = []
    trainY = []
    testX = []
    testY = []
    
    random.shuffle(raw[0])
    trainX = raw[0][0:int(len(raw[0])*train_test)]
    trainY = [0]*len(trainX)
    testX = raw[0][int(len(raw[0])*train_test):]
    testY = [0]*len(testX)
    
    for i in range(1,len(raw)):
        random.shuffle(raw[i])
        trainX.extend(raw[i][0:int(len(raw[i])*train_test)])
        trainY.extend([i]*int(len(raw[i])*train_test))
        testX.extend(raw[i][int(len(raw[i])*train_test):])
        testY.extend([i]*(len(raw[i])-(int(len(raw[i])*train_test))))

    return trainX, np.array(trainY).astype("float32"), testX, np.array(testY).astype("float32")

trainX, trainY, testX, testY = train_test_split(raw, train_test=0.7)

print(len(trainX))
print(trainY)
print(len(testX))
print(testY)

74
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3.]
35
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 2. 2. 2. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]


In [7]:
# Data Pre-processing
trainX = sequence.pad_sequences(trainX, maxlen=maxlen, dtype='float32')
testX = sequence.pad_sequences(testX, maxlen=maxlen, dtype='float32')

print('Train X shape:', trainX.shape)
print('Test X shape:', testX.shape)

Train X shape: (74, 512, 2)
Test X shape: (35, 512, 2)


In [8]:
trainY = to_categorical(trainY, num_fan_level)
testY = to_categorical(testY, num_fan_level)

print('Train Y shape:', trainY.shape)
print('Test Y shape:', testY.shape)

Train Y shape: (74, 4)
Test Y shape: (35, 4)


In [9]:
# LSTM Model
model = Sequential()
model.add(BatchNormalization(axis=1))
model.add(LSTM(4))
model.add(Dense(num_fan_level, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

print('Train...')
model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=64,
          validation_data=(testX, testY))

model.summary()

Train...
Epoch 1/64
5/5 [==============================] - 0s 64ms/step - loss: 1.4892 - accuracy: 0.0541 - val_loss: 1.5226 - val_accuracy: 0.0286
Epoch 2/64
5/5 [==============================] - 0s 16ms/step - loss: 1.4663 - accuracy: 0.0541 - val_loss: 1.5055 - val_accuracy: 0.0857
Epoch 3/64
5/5 [==============================] - 0s 16ms/step - loss: 1.4617 - accuracy: 0.0541 - val_loss: 1.4919 - val_accuracy: 0.1714
Epoch 4/64
5/5 [==============================] - 0s 16ms/step - loss: 1.4447 - accuracy: 0.1216 - val_loss: 1.4799 - val_accuracy: 0.1429
Epoch 5/64
5/5 [==============================] - 0s 16ms/step - loss: 1.4363 - accuracy: 0.1486 - val_loss: 1.4690 - val_accuracy: 0.1429
Epoch 6/64
5/5 [==============================] - 0s 14ms/step - loss: 1.4210 - accuracy: 0.1622 - val_loss: 1.4581 - val_accuracy: 0.1714
Epoch 7/64
5/5 [==============================] - 0s 14ms/step - loss: 1.4179 - accuracy: 0.1757 - val_loss: 1.4483 - val_accuracy: 0.1429
Epoch 8/64
5/5 [==

In [10]:
score, acc = model.evaluate(testX, testY,
                            batch_size=batch_size)

print('Test score:', score)
print('Test accuracy:', acc)

3/3 [==============================] - 0s 4ms/step - loss: 1.2101 - accuracy: 0.5143
Test score: 1.2101233005523682
Test accuracy: 0.5142857432365417
